In [1]:
# Loading
import pandas as pd

In [2]:
# Function to normalise for minimised variables
def normalisationmin(i):
    return (i-i.min())/(i.max()-i.min())

In [3]:
# Function to normalise for maximised variables
def normalisationmax(i):
    return (i-i.max())/(i.min()-i.max())

# User Input

# 1. Optimisation results

## 1.1 Import results

Import Pareto optimal results. 
Enter file name containing results, and number of rows to skip before headers using header = x. 1 row of headers only.

Data file given here is for the water grid case study documented in the accompanying paper, with a 5 year planning period, non-dominated results of 5 seeds, optimised using NSGA-II algorithm

In [4]:
opdata = pd.read_csv("Data/optimisation results 5 combined seeds.csv")

First column contains option number. Removing first column

In [5]:
opdata.drop(opdata.columns[0], axis = 1, inplace = True)

## 1.2 Cleanup data

Cleanup data headings: Remove $ and . from column headers

In [6]:
opdata.rename(columns = lambda x: x.replace('$', ''), inplace = True)
opdata.rename(columns = lambda x: x.replace('.', ''), inplace = True)

Convert units (select columns to divide by 1,000 or 1,000,000 to make more appropriate units.
Also fix negated objectives. 
Ensure to update column numbers when new results are input.
Headings changed below.

In [7]:
# convert storage to GL and absolute value
opdata[opdata.columns[16:18]] = abs(opdata[opdata.columns[16:18]]/1000)

# convert to $ million
opdata[opdata.columns[18:24]] = opdata[opdata.columns[18:24]]/1000000

# convert spill to GL
opdata[opdata.columns[24:27]] = opdata[opdata.columns[24:27]]/1000

Rename columns using list of replacement columns if desired. 
To print a list of column names for editing, use print(list(opdata.columns.values)) before replacing columns

In [8]:
print(list(opdata.columns.values))

['XTwoWayPipelinesNPI2NPI2Threshold', 'XTwoWayPipelinesNPI2NPI2FlowThreshold', 'XTwoWayPipelinesNPINPIThreshold', 'XTwoWayPipelinesNPINPIFlowThreshold', 'XTwoWayPipelinesBrisbanetoNthPineInterconnectorBrisbanetoNthPineThreshold', 'XTwoWayPipelinesBrisbanetoNthPineInterconnectorBrisbanetoNthPineFlowThreshold', 'XTwoWayPipelinesMaroochytoBaroonInterconnectorMaroochytoBaroonThreshold', 'XTwoWayPipelinesEwenMaddocktoBaroonInterconnectorEwenMaddocktoBaroonThreshold', 'XTwoWayPipelinesEPIEPIThreshold', 'XTwoWayPipelinesEPIEPIFlowThreshold', 'XTwoWayPipelinesSPISPIThreshold', 'XTwoWayPipelinesSPISPIFlowThreshold', 'XWCRWSPRWThreshold', 'XTugunDesalinationTugunDesalOneThirdThreshold', 'XTugunDesalinationTugunDesalTwoThirdsThreshold', 'XTugunDesalinationTugunDesalFullThreshold', 'XSpillVolumesTotalSpillVolume', 'XMinimumSystemStorageNegation', 'XCostsTotalCost', 'XCostsSwitchingCost', 'XCostsPumpingCost', 'XCostsTugunDesalCost', 'XCostsTreatmentCost', 'XCostsPRWCost', 'XTugunDesalinationTugunDe

In [9]:
replacementcolumns = ['NPI2 Threshold', 'NPI2 Flow Threshold', 'NPI Threshold', 'NPI Flow Threshold', 
                      'Brisbane to Nth Pine Threshold', 'Brisbane to Nth Pine Flow Threshold', 'Maroochy to Baroon Threshold', 
                      'Ewen Maddock to Baroon Threshold', 'EPI Threshold', 'EPI Flow Threshold', 'SPI Threshold', 
                      'SPI Flow Threshold', 'PRW Threshold', 'Desalination 1/3 Production Threshold', 
                      'Desalination 2/3 Production Threshold', 'Desalination Full Production Threshold', 
                      'Total Spill Volume (GL)', 'Minimum System Storage (GL)', 'Total Cost ($ million)', 
                      'Switching Cost ($ million)', 'Pumping Cost ($ million)', 'Desalination Cost ($ million)', 
                      'Treatment Cost ($ million)', 'PRW Cost ($ million)', 'Desalination Total Production Volume (GL)', 
                      'PRW Inflow (GL)', 'Two Way Pipeline Flow (GL)', 'Total Switch Count', 'Brisbane to Nth Pine Switch Count', 
                      'EPI Switch Count', 'Ewen to Baroon Switch Count', 'Maroochy to Baroon Switch Count', 'NPI2 Switch Count', 
                      'NPI Switch Count', 'SPI Switch Count', 'Environmental Flow Deficit', 'Total Volumetric Reliability']


if len(replacementcolumns) != len(opdata.columns):
    print 'replacement columns do not match length of original columns'

for i in range(len(opdata.columns)):
    opdata.rename(columns={opdata.columns[i]:replacementcolumns[i]}, inplace=True)

## 1.3 Save formatted Pareto set (inc. to pickle for use by other notebooks)

In [10]:
opdata.to_pickle("Data/opdata")

In [11]:
opdata.to_csv("Data/opdata.csv", index = False)

Extract Pareto set objective functions

In [12]:
obj1 = opdata['Minimum System Storage (GL)']
obj1name = obj1.name
obj2 = opdata['Total Cost ($ million)']
obj2name = obj2.name
obj3 = opdata['Total Spill Volume (GL)']
obj3name = obj3.name

Save Pareto set objective functions

In [13]:
objdict = {obj1name: obj1, obj2name: obj2, obj3name: obj3}

objdf = pd.DataFrame(objdict)

In [14]:
obj1.to_pickle("Data/obj1")
obj2.to_pickle("Data/obj2")
obj3.to_pickle("Data/obj3")

objdf.to_pickle("Data/objdf")

Extract and save Pareto set decision variables

In [15]:
# Enter titles/order of decision variables to create data frame
dvdf = pd.DataFrame(opdata, columns = ['NPI2 Threshold', 'NPI2 Flow Threshold', 'NPI Threshold', 'NPI Flow Threshold', 
                                        'Brisbane to Nth Pine Threshold', 'Brisbane to Nth Pine Flow Threshold', 
                                        'Maroochy to Baroon Threshold', 'Ewen Maddock to Baroon Threshold', 
                                        'EPI Threshold', 'EPI Flow Threshold', 'SPI Threshold', 
                                        'SPI Flow Threshold', 'PRW Threshold', 'Desalination 1/3 Production Threshold', 
                                        'Desalination 2/3 Production Threshold', 'Desalination Full Production Threshold',])

In [16]:
dvdf.to_pickle("Data/dvdf")

Normalise objectives and save

Normalisation with 0 as preferred/ideal value, 1 as least preferred/non-ideal

Used for decision maps, line diagram, radar chart and parallel coordinates

In [17]:
#Create new column/chart names for normalised objectives, with units removed and newlines
normcols = ['Minimum\nSystem Storage', 'Total Cost', 'Total\nSpill']

In [18]:
normobj1 = normalisationmax(obj1)
normobj2 = normalisationmin(obj2)
normobj3 = normalisationmin(obj3)

#create dataframe that preserves order of objectives
normobjdf = pd.DataFrame.from_items([(normcols[0], normobj1), (normcols[1], normobj2), (normcols[2], normobj3)])

In [19]:
normobjdf.to_pickle("Data/normobjdf")

1-normalise objectives and save

Normalisation with 1 as preferred/ideal value, 0 as least preferred/non-ideal

Used in compromise programming

In [20]:
onenormobj1 = normalisationmin(obj1)
onenormobj2 = normalisationmax(obj2)
onenormobj3 = normalisationmax(obj3)

#create dataframe that preserves order of objectives
onenormobjdf = pd.DataFrame.from_items([(obj1name, onenormobj1), 
                                        (obj2name, onenormobj2),
                                        (obj3name, onenormobj3)])

In [21]:
onenormobj1.to_pickle("Data/onenormobj1")
onenormobj2.to_pickle("Data/onenormobj2")
onenormobj3.to_pickle("Data/onenormobj3")

onenormobjdf.to_pickle("Data/onenormobjdf")

# 3. Import and format cluster results

## 3.1 Medoids

Enter objective function medoids (cluster representatives), from cluster analysis. 

For case study, cluster analysis was done using R. (see ClusterAnalysis.R)

There are 3 cluster sets for the case study, based on values of: objectives, decision variables, and objectives + decision variables respectively

In [22]:
#clusters based on objectives
medoids = pd.read_csv('Data/medoids.csv', header = 0)

In [23]:
#clusters based on decision variables
dvmedoids = pd.read_csv('Data/dvmedoids.csv', header = 0)

In [24]:
#clusters based on objectives and decision variables
allmedoids = pd.read_csv('Data/allmedoids.csv', header = 0)

Create dataframe of objective function medoids

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [25]:
#clusters based on objectives

medobj1 = medoids['Minimum System Storage (GL)']
medobj2 = medoids['Total Cost ($ million)']
medobj3 = medoids['Total Spill Volume (GL)']

medobjdf = pd.DataFrame.from_items([(medobj1.name, medobj1), (medobj2.name, medobj2), (medobj3.name, medobj3)])

# Identify option number (index) of medoids within wider Pareto set
medindex = medoids['Option']-1 # converting R index of medoids (starts at 1) to python index

In [26]:
#clusters based on decision variables

# Identify option number (index) of medoids within wider Pareto set
dvmedindex = dvmedoids['Option']-1 # converting R index of medoids (starts at 1) to python index

dvmedobj1 = opdata.ix[dvmedindex,'Minimum System Storage (GL)']
dvmedobj2 = opdata.ix[dvmedindex,'Total Cost ($ million)']
dvmedobj3 = opdata.ix[dvmedindex,'Total Spill Volume (GL)']

dvmedobjdf = pd.DataFrame.from_items([(dvmedobj1.name, dvmedobj1), 
                                      (dvmedobj2.name, dvmedobj2), 
                                      (dvmedobj3.name, dvmedobj3)])

In [27]:
#clusters based on normalised objectives and decision variables combined

allmedobj1 = allmedoids['Minimum System Storage (GL)']
allmedobj2 = allmedoids['Total Cost ($ million)']
allmedobj3 = allmedoids['Total Spill Volume (GL)']

allmedobjdf = pd.DataFrame.from_items([(allmedobj1.name, allmedobj1), 
                                       (allmedobj2.name, allmedobj2), 
                                       (allmedobj3.name, allmedobj3)])

# Identify option number (index) of medoids within wider Pareto set
allmedindex = allmedoids['Option']-1 # converting R index of medoids (starts at 1) to python index

Save objective function medoids to pickle for use by other notebooks

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [28]:
medobj1.to_pickle("Data/medobj1")
medobj2.to_pickle("Data/medobj2")
medobj3.to_pickle("Data/medobj3")

medobjdf.to_pickle("Data/medobjdf")

medindex.to_pickle("Data/medindex")

In [29]:
dvmedobj1.to_pickle("Data/dvmedobj1")
dvmedobj2.to_pickle("Data/dvmedobj2")
dvmedobj3.to_pickle("Data/dvmedobj3")

dvmedobjdf.to_pickle("Data/dvmedobjdf")

dvmedindex.to_pickle("Data/dvmedindex")

In [30]:
allmedobj1.to_pickle("Data/allmedobj1")
allmedobj2.to_pickle("Data/allmedobj2")
allmedobj3.to_pickle("Data/allmedobj3")

allmedobjdf.to_pickle("Data/allmedobjdf")

allmedindex.to_pickle("Data/allmedindex")

## 3.2 Normalise medoids

Normalise medoids, with 0 being preferred value, 1 least preferred 

Relative to max or min, need to change equations below to suit maximum or minimised objectives

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [31]:
normmedobj1 = (medobj1-obj1.max())/(obj1.min()-obj1.max()) #maximised obj
normmedobj2 = (medobj2-obj2.min())/(obj2.max()-obj2.min()) #minimised obj
normmedobj3 = (medobj3-obj3.min())/(obj3.max()-obj3.min()) #minimised obj

In [32]:
normdvmedobj1 = (dvmedobj1-obj1.max())/(obj1.min()-obj1.max()) #maximised obj
normdvmedobj2 = (dvmedobj2-obj2.min())/(obj2.max()-obj2.min()) #minimised obj
normdvmedobj3 = (dvmedobj3-obj3.min())/(obj3.max()-obj3.min()) #minimised obj

In [33]:
normallmedobj1 = (allmedobj1-obj1.max())/(obj1.min()-obj1.max()) #maximised obj
normallmedobj2 = (allmedobj2-obj2.min())/(obj2.max()-obj2.min()) #minimised obj
normallmedobj3 = (allmedobj3-obj3.min())/(obj3.max()-obj3.min()) #minimised obj

Format normalised medoid data into dataframe

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [34]:
normmedoidobjdf = pd.DataFrame({obj1name: normmedobj1, obj2name: normmedobj2,
                         obj3name: normmedobj3})

In [35]:
normdvmedoidobjdf = pd.DataFrame({obj1name: normdvmedobj1, obj2name: normdvmedobj2,
                         obj3name: normdvmedobj3})

In [36]:
normallmedoidobjdf = pd.DataFrame({obj1name: normallmedobj1, obj2name: normallmedobj2,
                         obj3name: normallmedobj3})

Save to pickle for use by other scripts

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [37]:
normmedoidobjdf.to_pickle("Data/normmedoidobjdf")

In [38]:
normdvmedoidobjdf.to_pickle("Data/normdvmedoidobjdf")

In [39]:
normallmedoidobjdf.to_pickle("Data/normallmedoidobjdf")

Normalise medoid decision variables

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [40]:
normmedoiddvdf = pd.DataFrame(index = range(0, len(medindex)), columns = dvdf.columns)
for i in range(0, len(medindex)):
    for j in dvdf.columns:
        normmedoiddvdf.ix[i,j] = (dvdf.ix[medindex[i],j]-dvdf[j].min())/(dvdf[j].max()-dvdf[j].min())

In [41]:
normdvmedoiddvdf = pd.DataFrame(index = range(0, len(dvmedindex)), columns = dvdf.columns)
for i in range(0, len(dvmedindex)):
    for j in dvdf.columns:
        normdvmedoiddvdf.ix[i,j] = (dvdf.ix[dvmedindex[i],j]-dvdf[j].min())/(dvdf[j].max()-dvdf[j].min())

In [42]:
normallmedoiddvdf = pd.DataFrame(index = range(0, len(allmedindex)), columns = dvdf.columns)
for i in range(0, len(allmedindex)):
    for j in dvdf.columns:
        normallmedoiddvdf.ix[i,j] = (dvdf.ix[allmedindex[i],j]-dvdf[j].min())/(dvdf[j].max()-dvdf[j].min())

Create dataframe of combined normalised objective functions and decision variables of medoids and save to pickle

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [43]:
normmedoiddf = pd.concat([normmedoiddvdf, normmedoidobjdf], axis = 1)
normmedoiddf.to_pickle("Data/normmedoiddf")

In [44]:
normdvmedoiddf = pd.concat([normdvmedoiddvdf, normdvmedoidobjdf], axis = 1)
normdvmedoiddf.to_pickle("Data/normdvmedoiddf")

In [45]:
normallmedoiddf = pd.concat([normallmedoiddvdf, normallmedoidobjdf], axis = 1)
normallmedoiddf.to_pickle("Data/normallmedoiddf")

## 3.3 Cluster mapping (cluster membership of each option)

Import cluster mapping: identifies cluster membership of each decision option in the Pareto set. 

Rows should correspond to the same options as the opdata.

Save to pickle.

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [46]:
clusternos = pd.read_csv('Data/clusternos.csv', usecols = [1], header = 0, names = ['Cluster']) 
    #importing cluster numbers from R results. This needs to be updated when opdata is updated
clusternos.to_pickle("Data/clusternos")

In [47]:
dvclusternos = pd.read_csv('Data/dvclusternos.csv', usecols = [1], header = 0, names = ['Cluster']) 
    #importing cluster numbers from R results. This needs to be updated when opdata is updated
dvclusternos.to_pickle("Data/dvclusternos")

In [48]:
allclusternos = pd.read_csv('Data/allclusternos.csv', usecols = [1], header = 0, names = ['Cluster']) 
    #importing cluster numbers from R results. This needs to be updated when opdata is updated
allclusternos.to_pickle("Data/allclusternos")

## 3.4 Set cluster colours for plotting

Create colour pool for clusters (add colours to taste, depending on number of clusters)

Using this pool allows consistency between plots

This clusterpool can be used for all 3 cluster sets of the case study

In [49]:
clustcols = pd.Series(['brown', 'red', 'magenta', 'yellow', 'aqua', 'Green', 
          'SteelBlue', 'Navy', 'purple', 'Gray', 'Black'], name = 'Cluster Colours')

In [50]:
clustcols.to_pickle("Data/clustercolours")

Create list of colours for plotting, one for each decision option/index. Ensure colorpool is applied in the same order to each option/cluster.

For each of the 3 cluster sets (objectives, decision variables, objectives + decision variables)

In [57]:
clustcollist = []
for i in clusternos.index:
    clustcollist.append(clustcols[clusternos['Cluster'][i]-1])
clustcolseries = pd.Series(clustcollist, name = 'Cluster Colour List')

clustcolseries.to_pickle("Data/clustcollist")

In [52]:
dvclustcollist = []
for i in dvclusternos.index:
    dvclustcollist.append(clustcols[dvclusternos['Cluster'][i]-1])
dvclustcolseries = pd.Series(dvclustcollist, name = 'Cluster Colour List')

dvclustcolseries.to_pickle("Data/dvclustcollist")

In [53]:
allclustcollist = []
for i in allclusternos.index:
    allclustcollist.append(clustcols[allclusternos['Cluster'][i]-1])
allclustcolseries = pd.Series(allclustcollist, name = 'Cluster Colour List')

allclustcolseries.to_pickle("Data/allclustcollist")

## 3.5 Set cluster markers for plotting

Create marker pool for clusters (add markers to taste, depending on number of clusters)

Using this pool allows consistency between plots, and allows plots to be presented in black and white

For cluster set 1 only, since this is the only one presented in the paper

In [75]:
clustmarks = pd.Series(["o","v","^","<",">","D","s","p","*","h",
              "+","x","8"], name = "Cluster Markers")

In [76]:
clustmarks.to_pickle("Data/clustermarkers")

Create list of markers for plotting, one for each decision option/index. Ensure markerpool is applied in the same order to each option/cluster.

In [77]:
clustmarklist = []
for i in clusternos.index:
    clustmarklist.append(clustmarks[clusternos['Cluster'][i]-1])
clustmarkseries = pd.Series(clustmarklist, name = 'Cluster Marker List')

clustmarkseries.to_pickle("Data/clustmarklist")

# 4. Create shortlist of options based on results visual and post-optimisation analysis

## Enter row numbers of shortlist for selecting shortlisted solution
Remember in Python they start at 0

In [14]:
shortlist_rows = [medindex[2-1], medindex[9-1], medindex[10-1], obj2.idxmin(), 471, 672, 670, 295, 348]

In [15]:
print shortlist_rows

[218, 509, 138, 405, 471, 672, 670, 295, 348]


## Create category to identify shortlist and save to pickle

In [46]:
shortlist_decider = lambda value: "Option %d" % (x+1) if value in shortlist_rows else "Other option"
shortlist = pd.DataFrame({'Shortlist':[shortlist_decider(x) for x in xrange(len(opdata))]})

In [47]:
shortlist.to_pickle("Data/shortlist")

## Create a dataframe of shortlist data and save to pickle

In [48]:
shortlistdf = pd.DataFrame(opdata.ix[shortlist_rows])

In [49]:
shortlistdf.to_pickle("Data/shortlistdf")

## Save to csv, with decision options as columns instead of rows and converted to start from 1 (instead of python 0-index). Used for presentation purposes.

In [53]:
shortlistdftf = pd.DataFrame(index = shortlistdf.columns, columns = shortlistdf.index+1)
for i in shortlistdf.columns:
    for j in shortlistdf.index:
        shortlistdftf.loc[i,j+1] = str("%.2f") % shortlistdf.loc[j,i]
shortlistdftf.to_csv("Data/shortlist.csv")

## Normalise shortlist objective functions and save to pickle

In [54]:
normshortobj1 = (shortlistdf['Minimum System Storage (GL)']-obj1.max())/(obj1.min()-obj1.max())
normshortobj2 = (shortlistdf['Total Cost ($ million)']-obj2.min())/(obj2.max()-obj2.min())
normshortobj3 = (shortlistdf['Total Spill Volume (GL)']-obj3.min())/(obj3.max()-obj3.min())

In [55]:
normshortlistobjdf = pd.DataFrame.from_items([(obj1name, normshortobj1), 
                                           (obj2name, normshortobj2), 
                                           (obj3name, normshortobj3)])

In [56]:
normshortlistobjdf.to_pickle("Data/normshortlistobjdf")